In [1]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
import os

from coeditor.dataset import TokenizedEditDataset, is_repetitive_edit
from coeditor.model import CoeditorModel, _Tokenizer, DecodingArgs, EvalArgs
from coeditor.history import show_change
from coeditor.encoding import TokenizedEdit, decode_tokens, WindowArgs, tokens_to_change
import shutil
import random

os.chdir(proj_root())

In [2]:
n_samples = 50
wargs = WindowArgs(4096)

test_data_name = "small"
test_data: TokenizedEditDataset = pickle_load(
    get_dataset_dir(test_data_name) / "tokenized-file_based" / "test.pkl"
)
test_data = test_data.map(lambda e: e.truncate_ctx(wargs))

rep_edits = pfilter(is_repetitive_edit, test_data.all_edits())
print("Total edits: ", len(list(test_data.all_edits())))
print("Repetitive edits: ", len(rep_edits))

# random.seed(123)
# random.shuffle(rep_edits)
# rep_edits = rep_edits[:n_samples]


filtering: 100%|██████████| 2186/2186 [00:00<00:00, 3340.42it/s]


Total edits:  2186
Repetitive edits:  210


In [3]:
model_dir = get_model_dir(trained=True) / "small"
# model_dir = get_model_dir(trained=True) / "coeditor-small-skip"
model = CoeditorModel.load_pretrained(model_dir)
model.to("cuda:1")

eval_args = EvalArgs(4096 * 2, WindowArgs(4096))


In [4]:
rep_data = TokenizedEditDataset.from_edits(rep_edits)
rep_result = model.eval_on_data(rep_data, eval_args)
display(rep_result)


evaluate loss: 100%|██████████| 68/68 [00:37<00:00,  1.83batch/s]


{'loss_per_ex': (mean=5.2627, weight=210),
 'loss_per_tk': (mean=0.11456, weight=9647),
 'prob_per_ex': (mean=0.3824, weight=210)}

In [19]:
all_result = model.eval_on_data(test_data, eval_args)
display(all_result)


evaluate loss: 100%|██████████| 663/663 [05:32<00:00,  1.99batch/s]


{'loss_per_ex': (mean=41.051, weight=2398),
 'loss_per_tk': (mean=0.76586, weight=128535),
 'prob_per_ex': (mean=0.035363, weight=2398)}

In [5]:
out_dir = Path("output/inspect_coeditor") / model_dir.name
shutil.rmtree(out_dir, ignore_errors=True)
(out_dir / "correct").mkdir(parents=True, exist_ok=True)
(out_dir / "incorrect").mkdir(parents=True, exist_ok=True)

decode_args = DecodingArgs()
exact_match = WeightedSum(0, 0)
predictions = []

for i, ex in enumerate(tqdm(rep_edits)):
    pred_tks = model.predict(ex.input_tks, decode_args)
    predictions.append(pred_tks)
    pred_change = TokenizedEdit(ex.path, ex.input_tks, pred_tks).as_change(True)
    truth_change = ex.as_change(True)
    is_correct = normalize_code_by_ast(pred_change.after) == normalize_code_by_ast(truth_change.after)
    exact_match += WeightedSum(int(is_correct), 1)

    compare_str = ex.show_prediction(pred_tks)
    out_file = out_dir / ("correct" if is_correct else "incorrect") / f"ex-{i}.txt"
    out_file.write_text(compare_str)

print("Exact match: ", exact_match.average())


100%|██████████| 210/210 [05:53<00:00,  1.68s/it]

Exact match:  0.49523809523809526
